In [9]:
import pickle
import numpy as np
import nltk
from nltk.translate import bleu_score 
from collections import Counter
from collections import defaultdict
import string
import copy
#!pip install pronouncing
import pronouncing
import numpy as np
#import os
import glob
import csv

In [10]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno 8] nodename nor
[nltk_data]     servname provided, or not known>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 8] nodename nor servname provided, or not
[nltk_data]     known>


False

In [3]:
# some basic preprocessing techniques
def prepString(s):
    '''removes punctuation other than apostrophes from string'''
    return str(s).lower().translate({ord(c): None for c in string.punctuation if c not in ("'")})

def removePunc(s):
    '''removes punctuation from string'''
    return str(s).lower().translate({ord(c): None for c in string.punctuation})

def removeMarkupWords(s):
    '''removes positional words generated in lyrics'''
    s = str(s).lower()
    for term in ['xeol','xbol','xeos','xbos','[verse-1]','[verse-2]','[chorus]']:
        s = str(s).replace(term,'')
    return s

In [11]:
exampleWord = 'orange'
examplePhones = pronouncing.phones_for_word(exampleWord)[0]
exampleRhymePart = pronouncing.rhyming_part(examplePhones)

In [12]:
examplePhones

'AO1 R AH0 N JH'

In [13]:
exampleRhymePart

'AO1 R AH0 N JH'

In [14]:
def calcRhymeDensity(text,rhymeType='perfect',rhymeLocation='all',lineStartStop=(1,-2),printExamples=False):
    '''calculates rhyme density (count of rhymes over n-1 words). \n\n
    
       _parameters_
       text: input text for measurement
       rhymeType: 'perfect' is a perfect rhyme, 'vowel' is a rhyming in the vowel sound + stress only
       rhymeLocation: choose to look at 'all' text, 'section' by line numbers, or 'end' (last word in each line)    
       lineStartStop: tuple of (start,stop) line numbers
       printExamples: if True, print most common values of the selected rhymeType
       
       _returns_
       rhyme_cnt: count of rhymes of specified rhymeType and rhymeLocation
       wordCount: count of words of specified rhymeType and rhymeLocation
       rhymeDensity: rhyme_cnt/float(wordCount-1)
    '''
    # restrict location to (end=last word, internal line = line, all= full text)
    # count tokens
    # 
    
    # initialize
    rhymePart_cnt = Counter()
    rhyme_cnt = 0
    
    # prepare data
    text = prepString(removeMarkupWords(text))
    
    if rhymeLocation == 'all':
        words = text.split()
    
    if rhymeLocation == 'end':
        lines = text.split("\n")
        words = [line.split()[-1] for line in lines if len(line.split())>0]
    
    if rhymeLocation == 'section':
        lines = text.split("\n")
        words = [line.split()[-1] for line in lines[lineStartStop[0]:lineStartStop[1]+1] if len(line.split())>0]
    
    # 
    wordCount = len(words)
    #print(words)
    for word in words:
        pros = pronouncing.phones_for_word(word)
        if pros:     
            phonelist = pros[0]  #using first pronunciation for now
            if len(phonelist) > 0:
                if rhymeType == 'perfect':
                    rhymePart_cnt[pronouncing.rhyming_part(phonelist)] +=1

                #if rhymeType == 'rime':
                #    pass
                #if rhymeType == 'soft':
                #    pass
                #if rhymeType == 'consonant':
                #    pass

                elif rhymeType == 'vowel':
                    rhymePart_cnt[pronouncing.rhyming_part(phonelist).split()[0]] +=1
    
    for v in rhymePart_cnt.values():
        rhyme_cnt += v-1
    
    if wordCount>1: 
        rhymeDensity = rhyme_cnt/float(wordCount-1)
    else:
        rhymeDensity = 0.0
    
    if printExamples == True:
        print(rhymePart_cnt.most_common(5))
    
    return rhymeDensity, rhyme_cnt, wordCount

In [25]:
phoneText = "as I walk through the valley of the garden of death"
newText = "I take a look at my life and realize there's nothing left"
hickoryText = "Hickory Dickory Dock,\nThe mouse ran up the clock.\nThe clock struck one,\nThe mouse ran down!\nHickory Dickory Dock."

calcRhymeDensity([phoneText, newText], rhymeType='vowel', rhymeLocation='end', printExamples=True)

[('EH1', 1)]


(0.0, 0, 1)

In [26]:
prepString(removeMarkupWords([phoneText, newText]))

"'as i walk through the valley of the garden of death' i take a look at my life and realize there's nothing left"

In [60]:
zz = pronouncing.phones_for_word('pandora')[0]

In [61]:
pronouncing.rhyming_part(zz).split()

['AO1', 'R', 'AH0']